# Test of New Analysis Routine with Improved Offsets and Internal Resistances Included

In [2]:
%matplotlib tk
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.core.fitters as fts
import flopter.magnum.magoptoffline as mg
import analysis.scripts.magnum.magnum_postrun_initial_analysis as mpia

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)


In [3]:
mpia.aia_mapping_wrapper(157)


 Analysing shot 157...
Attempting analysis on shot 157
"2019-06-03_Leland/" 		 "2019-06-03 16h 02m 56s TT_06698304580043636299.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


0: 1996, 1: 1997
<xarray.Dataset>
Dimensions:     (direction: 2, sweep: 997, time: 50)
Coordinates:
  * time        (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0047 0.0048 0.0049
  * direction   (direction) <U4 'up' 'down'
    probe       <U1 'S'
Dimensions without coordinates: sweep
Data variables:
    voltage     (direction, sweep, time) float64 -94.69 -92.51 ... 7.573 9.753
    current     (direction, sweep, time) float64 -0.07715 -0.07801 ... 0.001293
    shot_time   (direction, sweep, time) float64 1.024 1.024 ... 10.99 10.99
    start_time  (direction, sweep) float64 1.024 1.034 1.044 ... 10.98 10.99
<xarray.Dataset>
Dimensions:     (time: 32)
Coordinates:
  * time        (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0029 0.003 0.0031
    probe       <U1 'S'
Data variables:
    voltage     (time) float64 -94.09 -91.97 -89.85 ... -32.49 -30.45 -28.48
    current     (time) float64 -0.05752 -0.05682 -0.05656 ... -0.03195 -0.01813
    shot_time   (time) float64 6.009 6.009 6.009 6

## Comparison of analysis with Old and New Magoptofflines
Seeing the difference between including the updated resistances and offsets

In [11]:
path_to_datasets = '/home/jleland/Data/external/magnum/'
path_to_analysed_datasets = 'analysed_*/a158*nc'
os.chdir(path_to_datasets)

analysed_dataset_fns = glob.glob(path_to_analysed_datasets)
analysed_dataset_fns.sort()
print(analysed_dataset_fns)

analysed_infos = []

for i, anal_ds in enumerate(analysed_dataset_fns):
    match = re.search("\/a{1}([0-9]{3})_([0-9]{3})_([0-9]{19})\.nc", anal_ds)
    shot_index = int(match.group(1))
    adc_index = int(match.group(2))
    shot_timestamp = int(match.group(3))
    shot_time = ut.human_datetime_str(int(match.group(3)))
    
    ds = xr.open_dataset(anal_ds)
    time_len = len(ds['time'])
    sweep_len = len(ds['sweep'])
    
    analysed_infos.append({
        'adc_index':adc_index, 
        'shot_number':shot_index, 
        'shot_timestamp':shot_timestamp, 
        'shot_time':shot_time, 
        'filename':anal_ds,
        'time_len': time_len,
        'sweep_len': sweep_len
    })

analysed_infos_df = pd.DataFrame(analysed_infos)

['analysed_1/a158_163_6698304817900493824.nc', 'analysed_2/a158_163_6698304817900493824.nc', 'analysed_3/a158_163_6698304817900493824.nc', 'analysed_3_downsampled/a158_163_6698304817900493824.nc']


In [12]:
analysed_infos_df

,adc_index,filename,shot_number,shot_time,shot_timestamp,sweep_len,time_len
0,163,analysed_1/a158_163_6698304817900493824.nc,158,2019-06-03 15:03:42,6698304817900493824,999,5000
1,163,analysed_2/a158_163_6698304817900493824.nc,158,2019-06-03 15:03:42,6698304817900493824,999,50
2,163,analysed_3/a158_163_6698304817900493824.nc,158,2019-06-03 15:03:42,6698304817900493824,999,5000
3,163,analysed_3_downsampled/a158_163_66983048179004...,158,2019-06-03 15:03:42,6698304817900493824,999,50


In [6]:
os.chdir('/home/jleland/Data/external/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

In [13]:
indices = [1, 3]

In [14]:
def preprocess_sel(ds, sweep=slice(0,650)):
    return ds.reset_index('time', drop=True).sel(sweep=sweep)

def preprocess_average(ds):
    ds = ds.reset_index('time', drop=True)
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

def preprocess(ds):
    return ds.reset_index('time', drop=True)

In [15]:
files_oi = analysed_infos_df.loc[indices]['filename'].values
analysis_type = xr.DataArray(['old', 'new'], dims=['analysis_type'], name='analysis_type')
analysis_type

<xarray.DataArray 'analysis_type' (analysis_type: 2)>
array(['old', 'new'], dtype='<U3')
Dimensions without coordinates: analysis_type

In [16]:
combined_ds = xr.open_mfdataset(files_oi, concat_dim=analysis_type, preprocess=preprocess_sel)
combined_ds = combined_ds.load()
combined_ds

<xarray.Dataset>
Dimensions:        (analysis_type: 2, direction: 2, probe: 2, sweep: 650, time: 50)
Coordinates:
  * direction      (direction) object 'up' 'down'
  * probe          (probe) object 'S' 'L'
  * analysis_type  (analysis_type) <U3 'old' 'new'
Dimensions without coordinates: sweep, time
Data variables:
    voltage        (analysis_type, probe, direction, sweep, time) float64 -95.2 ... 9.622
    current        (analysis_type, probe, direction, sweep, time) float64 -0.08749 ... 0.08398
    shot_time      (analysis_type, probe, direction, sweep, time) float64 1.004 ... 7.499
    start_time     (analysis_type, probe, direction, sweep) float64 1.004 ... 7.499

In [17]:
flattop = slice(0,650)

fig, ax = plt.subplots(2)
combined_ds.sel(probe='S', analysis_type='old').mean('direction').max('time')['current'].plot.line(ax=ax[0])
combined_ds.sel(probe='S', analysis_type='old', sweep=flattop).mean('direction').max('time')['current'].plot.line(ax=ax[0])
combined_ds.sel(probe='L', analysis_type='old').mean('direction').max('time')['current'].plot.line(ax=ax[1])
combined_ds.sel(probe='L', analysis_type='old', sweep=flattop).mean('direction').max('time')['current'].plot.line(ax=ax[1])
fig.show()

In [18]:
ds_avg = combined_ds.mean(['sweep', 'direction'])
ds_avg = ds_avg.assign({'d_current': combined_ds.std(['sweep', 'direction'])['current']})
ds_avg

<xarray.Dataset>
Dimensions:        (analysis_type: 2, probe: 2, time: 50)
Coordinates:
  * probe          (probe) object 'S' 'L'
  * analysis_type  (analysis_type) <U3 'old' 'new'
Dimensions without coordinates: time
Data variables:
    voltage        (analysis_type, probe, time) float64 -94.3 -92.19 ... 8.803
    current        (analysis_type, probe, time) float64 -0.0837 ... 0.07322
    shot_time      (analysis_type, probe, time) float64 4.254 4.254 ... 4.254
    start_time     (analysis_type, probe) float64 4.251 4.251 4.251 4.251
    d_current      (analysis_type, probe, time) float64 0.007994 ... 0.01781

In [27]:
fig, ax = plt.subplots(2)
ds_avg.set_coords('voltage').sel(probe='S')['current'].plot.line(x='voltage', hue='analysis_type', ax=ax[0])
ds_avg.set_coords('voltage').sel(probe='L')['current'].plot.line(x='voltage', hue='analysis_type', ax=ax[1])
fig.show()

In [20]:
def iv_fit(I, V, sigma, fit_above_vp=0.0):
    fitting_region = np.where(I >= (0 + fit_above_vp))
    print(I.shape, V.shape, sigma.shape)
    I = I[fitting_region]
    V = V[fitting_region] 
    sigma = sigma[fitting_region]
    
    fitter = fts.FullIVFitter()
    fit_data = fitter.fit(V, I, sigma=sigma)
    return fit_data

In [21]:
def _multi_fit_1d(I, V, t, sigma):
    print(I.shape, V.shape, sigma.shape)
#     iv_data = iv.IVData(V, I, t, sigma=sigma, estimate_error_fl=False)
    fit_data = iv_fit(I, V, sigma)
    return fit_data
    
#     temp = np.array(fit_data.get_temp())
#     d_temp = np.array(fit_data.get_temp_err())
#     isat = np.array(fit_data.get_isat())
#     d_isat = np.array(fit_data.get_isat_err())
#     sexp = np.array(fit_data.get_sheath_exp())
#     d_sexp = np.array(fit_data.get_sheath_exp_err())
#     vf =  np.array(fit_data.get_param_val('V_f'))
#     d_vf =  np.array(fit_data.get_param_err('V_f'))
#     chi2 = np.array(fit_data.chi2)
#     r_chi2 = np.array(fit_data.reduced_chi2)
#     return temp, d_temp, isat, d_isat, sexp, d_sexp, vf, d_vf, chi2, r_chi2

def _multi_fit(I, V, t, sigma):
    return np.apply_along_axis(_multi_fit_1d, -1, I, V, t, sigma)

def multi_fit(I, V, t, sigma, dims):
    return xr.apply_ufunc(_multi_fit_1d, I, V, t, sigma, 
                          input_core_dims=[['time'], ['time'], ['time'], ['time']], 
#                           output_core_dims=[['time'], [], [], [], [], [], [], [], [], [], []],
#                           vectorize=True,
#                           dask='parallelized'
                         )

def apply_multi_fit(ds, dims):
    #  * -1.059
    # * 0.946
    I = -ds['current'].values
    V = ds['voltage'].values * 0.946
    t = ds['shot_time'].values
    sigma = ds['d_current'].values
    return multi_fit(I, V, t, sigma, dims)
    

In [22]:
I = -ds_avg['current'].values[0]
V = ds_avg['voltage'].values[0]
t = ds_avg['shot_time'].values[0]
sigma = ds_avg['d_current'].values[0]

print(I.shape, V.shape, sigma.shape)
fit_data = _multi_fit_1d(I, V, t, sigma)

fit_data

(2, 50) (2, 50) (2, 50)
(2, 50) (2, 50) (2, 50)
(2, 50) (2, 50) (2, 50)


In [26]:
apply_multi_fit(ds_avg.sel(probe='S', analysis_type='old'), 'time').plot()

(50,) (50,) (50,)
(50,) (50,) (50,)


In [31]:
1/1.0581

0.9451795841209829

In [24]:
magnum_probes = lp.MagnumProbes()

# fig, ax = plt.subplots(2)
fit_datas = apply_multi_fit(ds_avg.sel(probe='S'), dims=['analysis_type'])
for i, fd in enumerate(fit_datas):
    fd.plot()
    print(fd.get_temp())
    print(magnum_probes['S'].get_density(fd.get_isat(), fd.get_temp(), alpha=np.radians(10)))
#     ax[i].plot(*fd.get_raw_plottables())
#     ax[i].plot(*fd.get_fit_plottables())

(2, 50) (2, 50) (2, 50)
(2, 50) (2, 50) (2, 50)


TypeError: 'IVFitData' object is not iterable

In [25]:
ds_avg['current']

<xarray.DataArray 'current' (analysis_type: 2, probe: 2, time: 50)>
array([[[-0.083699, -0.083494, -0.083217, -0.082638, -0.081969, -0.081668,
         -0.081679, -0.08128 , -0.080859, -0.080346, -0.079908, -0.079139,
         -0.078779, -0.07786 , -0.07767 , -0.077542, -0.077296, -0.076601,
         -0.07626 , -0.075824, -0.074996, -0.074584, -0.074001, -0.073176,
         -0.072532, -0.071402, -0.070112, -0.068011, -0.064357, -0.058403,
         -0.047145, -0.027278,  0.003615,  0.045177,  0.093677,  0.143575,
          0.19338 ,  0.234532,  0.263201,  0.280153,  0.289532,  0.29738 ,
          0.303618,  0.306637,  0.31114 ,  0.314462,  0.316858,  0.318942,
          0.319615,  0.322095],
        [-0.010192, -0.010106, -0.010063, -0.009742, -0.009759, -0.009554,
         -0.009543, -0.009302, -0.009306, -0.009207, -0.008994, -0.008954,
         -0.008949, -0.008666, -0.008627, -0.008553, -0.008526, -0.008163,
         -0.007967, -0.007992, -0.007943, -0.007579, -0.008009, -0.007694,


In [105]:
meta_data_ds.sel(shot_number=157).load()

<xarray.Dataset>
Dimensions:                 (ts_radial_pos: 46)
Coordinates:
    shot_number             int32 157
    ts_number               float64 nan
    ts_timestamp            float64 nan
    ts_time                 datetime64[ns] NaT
    adc_index               float64 162.0
    adc_time                datetime64[ns] 2019-06-03T15:02:46
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            object '2019-06-03 16h 02m 56s TT_06698304580043636299.adc'
    ts_density              (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_temperature          (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_density            (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_temperature        (ts_radial_pos) float64 nan nan nan ... nan nan nan
    adc_folder              object '2019-06-03_Leland/'
    adc_calibration_index   object '120'
    adc_4_probe             object 'S'
    adc_5_p